In [ ]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
from os.path import join

In [ ]:
region = 'borde_rural'

geojson_fp = join('..', '..', 'data', region, 'train-'+region+'.geojson')
image_fp = join('..', '..', 'data', region, region+'_ortho-cog.tif')
roof_dir = join('..', '..', 'data', region, 'roofs_train')

In [ ]:
with rasterio.open(image_fp) as src:
    profile = src.profile
    print(profile['crs'])
    print(src.profile)
    print(src.bounds)

## 1. Get an overview of the scenery

In [ ]:
with rasterio.open(image_fp) as src:
    oviews = src.overviews(1)
    oview = oviews[-1]
    print('Decimation factor = {}'.format(oview))
    thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview')

## 2. Cut out roofs and store as separate files

In [ ]:
import rasterio
from rasterio.mask import mask
from os import makedirs
from os.path import exists

Create subfolders for each material

In [ ]:
materials = ['healthy_metal', 'irregular_metal', 'concrete_cement', 'incomplete', 'other']
for mat in materials:
    directory = join(roof_dir, mat) 
    if not exists(directory):
        makedirs(directory)

Coordinates in GeoJSON label file need to be converted to CSR format of the image

In [ ]:
from pyproj import Proj, transform
outProj = Proj(init=profile['crs']) # CRS format of image
inProj = Proj(init='epsg:4326') # lat/lon coordinate format

#### Cutting out roofs
1. Extract the roof id and polygon coordinates from GeoJSON label file
2. Cut the polygon out of the image
3. Save the cut out image of the roof to file with id as name

In [ ]:
with open(geojson_fp) as geojson:
    geoms = json.loads(geojson.read())
    roofs = geoms['features']

for roof in roofs:
    roof_id = roof['id']
    roof_geom = roof['geometry']
    roof_material = roof['properties']['roof_material']    
    
    print(roof_id)
    if roof_geom['type'] == 'MultiPolygon':
        print("MULTIPOLYGON")
        continue
    else:
        coord = roof_geom['coordinates'][0]
        for c in coord:
            c[0], c[1] = transform(inProj, outProj, c[0], c[1])
    
    print(roof_geom)
    with rasterio.open(image_fp) as image:
        roof_image, roof_transform = mask(image, [roof_geom], filled=True, crop=True)
    #show(roof_image)
    roof_meta = image.meta.copy()
    
    # save the resulting raster
    roof_meta.update({"driver": "GTiff",
        "dtype": rasterio.uint8,
        "height": roof_image.shape[1],
        "width": roof_image.shape[2],
        "transform": roof_transform,
        "tiled": True,
        "compress": 'lzw'})
    roof_image_fp = join(roof_dir, roof_material, str(roof_id)+".tif")

    with rasterio.open(roof_image_fp, "w", **roof_meta) as dest:
        dest.write(roof_image)                             

All this can also happen using the module `preprocessing`

In [1]:
import preprocessing

In [ ]:
region = 'borde_rural'
preprocessing.preprocess_region(region)

7a3f2a10
7a1f731e
7a424ad8
7a3edc5e
7a303a6e
7a1eb2bc
7a2c2a64
7a2c92ba
7a28a24a
7a38a5b4
7a349f28
7a371a32
7a44a472
7a2fc05c
7a41c798
7a21eea0
7a31c942
7a2d66d6
7a2f33ee
7a38cd5a
7a38c0ee
7a2187f8
7a3b0aa2
7a339bdc
7a303cf8
7a3a2858
7a38b1c6
7a2f5d7e
7a372928
7a35666a
7a21761e
7a44556c
7a2618b8
7a2d7540
7a329c1e
7a29168a
7a203b6e
7a3ed916
7a3d5078
7a1d364e
7a33a5a0
7a4106e6
7a21dda2
7a310188
7a343e16
7a23a4fc
7a29049c
7a3fe0f4
7a36767c
7a3392ea
7a2d1596
7a31e350
7a2c1038
7a2e2ed6
7a34ce8a
7a3b4490
7a20c41c
7a2fe01e
7a2a4a28
7a3de1b4
7a327072
7a32d2e2
7a33a1ea
7a3ec0b6
7a42454c
7a350990
7a41d670
7a25b2ce
7a313bf8
7a3074a2
7a208b50
7a1d6bbe
7a407a64
7a41a984
7a43a108
7a1e9750
7a2f2f02
7a3537c6
7a2c0188
7a1ca422
7a298124
7a3415f8
7a2fee24
7a26797a
7a26ec7a
7a442a92
7a39f860
7a402726
7a1e6a96
7a1f7a12
7a3b9a3a
7a372004
7a2ecddc
7a2f509a
7a320e20
7a1fb86a
7a2393a4
7a236a1e
7a1f3066
7a2c8ef0
7a34c930
7a342f8e
7a3b0d5e
7a2408b6
7a204370
7a3cd44a
7a3cbe4c
7a21aee0
7a35461c
7a244e52
7a1db56a
7

In [ ]:
preprocessing.greetings()